In [19]:
import pandas as pd
import networkx as nx
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

try:
    from mpl_toolkits import basemap
    bm = basemap.Basemap()
except:
    bm = None

In [9]:
# Load the data in `citiesToCities.csv` into a table (you can write your own code or use the pandas library for
# data processing in python; see http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html for details)

In [10]:
#Load the data and rename the columns
c2c = pd.read_csv('citiesToCities.csv', names=[
        'departure_city', 'departure_lng', 'departure_lat', 'departure_country',
         'arrival_city', 'arrival_lng', 'arrival_lat', 'arrival_country',
         'num_routes', 'distance'], skiprows=1)

In [11]:
# Construct a directed, weighted graph of flights between airports (the library networkx provides convenient methods, 
# e.g. see https://networkx.github.io/documentation/latest/tutorial/tutorial.html)

In [12]:
#Create a container for city locations
cities = {}

#Create a graph
graph = nx.DiGraph()

#Iterate over all rows of the data file
for departure_city, departure_lng, departure_lat, departure_country,\
    arrival_city, arrival_lng, arrival_lat, arrival_country,\
    num_routes, distance in c2c.values:
        
    #Add city locations to the dictionary
    cities[departure_city] = departure_lng, departure_lat
    cities[arrival_city] = arrival_lng, arrival_lat
    
    #Add the link
    graph.add_edge(departure_city, arrival_city, weight=num_routes)
    
#How many nodes/edges are there?
print "Number of nodes: {}".format(graph.number_of_nodes())
print "Number of edges: {}".format(graph.number_of_edges())

#Convert to undirected
undirected_graph = graph.to_undirected()

Number of nodes: 2988
Number of edges: 30594


In [13]:
# Visualise the network you have constructed. Is it consistent with your intuition?

In [14]:
#Display the network
if bm:
    bm.drawcoastlines()
nx.draw_networkx_nodes(graph, cities, node_size=1, color='r')
nx.draw_networkx_edges(graph, cities, alpha=.02)
plt.xlim(-185, 185)
plt.ylim(-65, 85)

(-65, 85)

In [ ]:
# Use the graph to query the airports you can fly to from London

In [15]:
graph['London']

{'Aarhus': {'weight': 1},
 'Aberdeen': {'weight': 9},
 'Abu Dhabi': {'weight': 5},
 'Abuja': {'weight': 2},
 'Accra': {'weight': 2},
 'Addis Ababa': {'weight': 2},
 'Agadir': {'weight': 3},
 'Alesund': {'weight': 1},
 'Alghero': {'weight': 1},
 'Algier': {'weight': 2},
 'Alicante': {'weight': 8},
 'Alma-ata': {'weight': 2},
 'Almeria': {'weight': 1},
 'Amilcar Cabral': {'weight': 1},
 'Amman': {'weight': 4},
 'Amsterdam': {'weight': 10},
 'Ancona': {'weight': 1},
 'Antalya': {'weight': 1},
 'Antigua': {'weight': 2},
 'Antwerp': {'weight': 1},
 'Ashkhabad': {'weight': 1},
 'Athens': {'weight': 5},
 'Atlanta': {'weight': 7},
 'Aviles': {'weight': 1},
 'Bahrain': {'weight': 5},
 'Baku': {'weight': 2},
 'Baltimore': {'weight': 3},
 'Bangalore': {'weight': 1},
 'Bangkok': {'weight': 5},
 'Banjul': {'weight': 1},
 'Barcelona': {'weight': 9},
 'Bari': {'weight': 1},
 'Basel-Mulhouse-Freiburg': {'weight': 6},
 'Beijing': {'weight': 3},
 'Beirut': {'weight': 3},
 'Belfast': {'weight': 10},
 'Be

In [ ]:
# Which airport has the smallest/largest number of destination airports?

In [17]:
degree = graph.out_degree()
x = min(degree, key=degree.get)
print "Smallest number of destinations: {}".format(x)
x = max(degree, key=degree.get)
print "Largest number of destinations: {}".format(x)

Smallest number of destinations: Chipata
Largest number of destinations: London


In [18]:
# Which airport has the smallest/largest number of routes
strength = graph.out_degree(weight='weight')
x = min(strength, key=degree.get)
print "Smallest number of routes: {}".format(x)
x = max(strength, key=degree.get)
print "Largest number of routes: {}".format(x)

Smallest number of routes: Chipata
Largest number of routes: London


In [ ]:
# Are the in-degree (number of incoming flights from distinct airports) and the out-degree (number of outgoing 
# flights from distinct airports) correlated?

In [24]:
in_degree = graph.in_degree().values()
out_degree = graph.out_degree().values()

plt.scatter(in_degree, out_degree)
print "Pearson's R: {}; p-value: {}".format(
    *stats.pearsonr(in_degree, out_degree))

Pearson's R: 0.999683353189; p-value: 0.0


In [ ]:
# Develop a function that takes the location of a walker as input and returns the location of the walker at the next
# time step by randomly choosing a neighbour of the input location

In [38]:
def make_step(graph, airport):
    # Stay at the airport if there are no outgoing connections
    if len(graph[airport]) == 0:
        return airport
    #Get the neighbours and weights
    neighbours, weights = zip(*[(neighbour, data['weight']) for neighbour, data 
                                        in graph[airport].iteritems()])
    #Normalise the weights
    weights = np.array(weights, np.float) / np.sum(weights)
    #Grab a neighbour
    return np.random.choice(neighbours, p=weights)

make_step(graph, 'London')

'Riga'

In [ ]:
# Develop a function that can take multiple steps at a time

In [31]:
def make_steps(graph, airport, num_steps):
    path = [airport]
    #Make steps until we have a path that is long enough
    while len(path) <= num_steps:
        airport = make_step(graph, airport)
        path.append(airport)
    return path

make_steps(graph, 'London', 4)

['London', 'Wroclaw', 'Bergamo', 'Palermo', 'London']

In [ ]:
# Simulate a trajectory starting in London and visualise it

In [33]:
# Generate a trajectory
path = make_steps(graph, 'London', 4)
print path
#Convert the path to a list of edges
edges = zip(path, path[1:])
#Draw the airports and edges
if bm:
    bm.drawcoastlines()
nx.draw_networkx_edges(undirected_graph, cities, edgelist=edges)
nx.draw_networkx_nodes(undirected_graph, cities, node_size=1, color='r')
plt.xlim(-185, 185)
plt.ylim(-65, 85)

['London', 'Hamburg', 'London', 'Porto', 'Newark']


(-65, 85)

In [ ]:
# Run a simulation of 10,000 walkers starting in London and visualise the number of walkers at each airport after
# two time steps and 200 time steps

In [34]:
#Which airports is a passenger likely to be at after `k` steps
k = 2
num_runs=10000
airports = Counter([make_steps(graph, 'London', k)[-1] for _ in range(num_runs)])

In [35]:
if bm:
    bm.drawcoastlines()
nodes, frequency = zip(*airports.items())
nx.draw_networkx_nodes(graph, cities, nodelist=nodes, node_size=np.asarray(frequency)+10, alpha=.5)

In [39]:
#Which airports is a passenger likely to be at after `k` steps
k = 200
num_runs=10000
airports = Counter([make_steps(graph, 'London', k)[-1] for _ in range(num_runs)])

In [40]:
if bm:
    bm.drawcoastlines()
nodes, frequency = zip(*airports.items())
nx.draw_networkx_nodes(graph, cities, nodelist=nodes, node_size=np.asarray(frequency)+10, alpha=.5)